Blah de blah

In [1]:
# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import gc
import tempfile
import datetime
import time
import os
import sys
import ipyparallel as ipp
#from joblib import Parallel, delayed
#import multiprocessing
sys.path.append('..\\Modules')
import usle_cfactor_analysis
#import zonal_stats_ForLayerObject
import resample_raster
#from rasterstats import zonal_stats
import matplotlib.pyplot as plt
%matplotlib inline

#Path to intersected Subcatch/FUs shapefile
pathToShapefile = "D:/MW/spatial_RC7/ExportedCatchments-FUs Intersection.shp"
#pathToShapefile = "D:/MW/spatial_RC7/mw_intscfu_clean.shp"
#pathToShapefile = "D:/MW/spatial_RC7/badFUOnly.shp"

uniqueIDField = "UNQID"
catchmentField = "IntSCs"
FUField = "IntFUs"

FUsToInclude = ["Grazing Open","Grazing Forested","Forestry","Conservation"]

#Path to template raster (probably subcatchments or DEM)
rasterTemplateFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Output intersected Subcat/FU raster
#Will get a CSV of same name written 'mapping' each FU to a unique ID (the unique ID will be the raster values)
rasterOutputFile = "D:/MW/spatial_RC7/MW_USLE_IntSubCatFu.tif"

#Path to K raster
rasterKFile = "D:/MW/Ram_USLE/athena_data/mw_kfact.asc"

#Path to LS raster
rasterLSFile = "D:/MW/Ram_USLE/athena_data/mw_sfact.asc"

#Path to Fines raster
rasterFinesFile = "D:/MW/Ram_USLE/athena_data/mw_cssurf.asc"

#Path to Scald raster
rasterScaldFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Path to Cfactor directory
cFactorDIR = "D:\MW\Ram_USLE\skip"
#cFactorDIR = "D:\MW\Ram_USLE\skip\justone"

#Cfactor extension
cFactExt = "tif"

#cFactor Date naming pattern
cFactPattern = "xxxyyyymm"

#More Cfact settings
defaultCfact = 0.05
reqdValidProp = 0.20

#declare a couple of variables
IntSCFUStr = "IntSCFU"
MeanKLSCStr = "Mean_KLSC"
MeanKLSCFinesStr = "Mean_KLSC_Fines"
MeanKStr = "Mean_K"
MeanFinesStr = "Mean_Fines"
MeanCfactStr = "Mean_CFact"
MeanLSStr = "Mean_LS"
MeanScaldStr = "Mean_Scald"
AreaStr = "Area"
TimeStampStr = "Time_Stamp"
CellCountStr = "Cell_Count"

#Directory and filename constants for USLE parameterisation
USLE_DIR_KLSC = "KLSC";
USLE_DIR_KLSCFINE = "KLSC_Fines";
USLE_DIR_KLSC_CFACT = "Cfact";
USLE_FILE_KLSC = "USLE_KLSC_Total";
USLE_FILE_KLSCFINES = "USLE_KLSC_FinePerc";
USLE_FILE_KLS_CFACT = "CFactor";

rasterResultsDF = pandas.DataFrame(columns=[IntSCFUStr,TimeStampStr,MeanKLSCStr,MeanKLSCFinesStr,MeanKStr,MeanCfactStr,MeanFinesStr,MeanLSStr,MeanScaldStr,AreaStr,CellCountStr])
#rasterResultsDF

In [2]:
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()
pid_map

{0: 8772,
 1: 9312,
 2: 9308,
 3: 11352,
 4: 9900,
 5: 8936,
 6: 12036,
 7: 8992,
 8: 10720,
 9: 2924,
 10: 12112,
 11: 6684}

In [3]:
#load the template raster
templateRas = gdal.Open(rasterTemplateFile)
template_GeoTrans = templateRas.GetGeoTransform()
template_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [4]:
#Load K factor
kFactRas = gdal.Open(rasterKFile)
kfact_GeoTrans = kFactRas.GetGeoTransform()
kfact_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [5]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info matches template, excellent")
else:
    print("Kfactor raster resolution info didn't match template, attempted a resample")
    kFactRas = resample_raster.resample_raster_to_match_template(templateRas, kFactRas, os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kFactRas = resample_raster_to_match_template(templateRas, kFactRas, "D:/aa/tester.tif")
    kfact_GeoTrans = kFactRas.GetGeoTransform()

Kfactor raster resolution info matches template, excellent


In [6]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info now matches template, excellent")
    #kFactRas = None
    #kFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kfact_GeoTrans = kFactRas.GetGeoTransform()
else:
    print("Kfactor raster resolution still not the same as the template, you shouldn't proceed")
#zone_ds = None
#print("rows: " + str(kFactRas.RasterYSize) + " cols: " + str(kFactRas.RasterXSize))

Kfactor raster resolution info now matches template, excellent


In [7]:
#kFactRas.GetRasterBand(1).GetNoDataValue()
templateRas.GetRasterBand(1).GetNoDataValue()

# The line below would get the kfactor raster written to disk, but it may not be necessary
#kFactRas = None

#this stuf to draw up a sample of KFactor, have to chop off heaps of edges so that the range in the colorbar doesn't consider -9999
#kFactRasArray = kFactRas.ReadAsArray()
#array_chop = kFactRasArray[3500:-3500,1500:-1500]  # chop off the outer specified pixels
#plt.imshow(array_chop, interpolation='nearest')
#plt.colorbar()
#plt.title('K Factor')
#plt.show()

-9999.0

In [8]:
#Load LS factor
lsFactRas = gdal.Open(rasterLSFile)
lsFact_GeoTrans = lsFactRas.GetGeoTransform()


In [9]:
if (lsFact_GeoTrans == template_GeoTrans):
    print("LSfactor raster resolution info matches template, excellent")
else:
    print("LSfactor raster resolution info didn't match template, attempted a resample")
    lsFactRas = resample_raster.resample_raster_to_match_template(templateRas, lsFactRas, os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
    lsFact_GeoTrans = lsFactRas.GetGeoTransform()

LSfactor raster resolution info didn't match template, attempted a resample


In [10]:
if (lsFact_GeoTrans == template_GeoTrans):
    print("LSfactor raster resolution info now matches template, excellent")
    #lsFactRas = None
    #lsFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
    #lsfact_GeoTrans = lsFactRas.GetGeoTransform()
else:
    print("LSfactor raster resolution still not the same as the template, you shouldn't proceed")

LSfactor raster resolution info now matches template, excellent


In [11]:
#Load Fines raster
FinesRas = gdal.Open(rasterFinesFile)
fines_GeoTrans = FinesRas.GetGeoTransform()
fines_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [12]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info matches template, excellent")
else:
    print("Fines raster resolution info didn't match template, attempted a resample")
    FinesRas = resample_raster.resample_raster_to_match_template(templateRas, FinesRas, os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    fines_GeoTrans = FinesRas.GetGeoTransform()

Fines raster resolution info matches template, excellent


In [13]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info now matches template, excellent")
    #FinesRas = None
    #FinesRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    #fines_GeoTrans = FinesRas.GetGeoTransform()
else:
    print("Fines raster resolution info still not the same as the template, you shouldn't proceed")

Fines raster resolution info now matches template, excellent


In [14]:
#Open shapefile
basePolys = ogr.Open(pathToShapefile)
type(basePolys)
nlay = basePolys.GetLayerCount()
lyr = basePolys.GetLayer(0)
ext = lyr.GetExtent()
#nlay
#ext
lyr.GetFeatureCount()
#plt.imshow(lyr)

1286

In [15]:
featCount = 0
for feat in lyr:
    ##print("Doing something")
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    featCount += 1
    if (featCount == 20):
        break

In [16]:
#FUsToInclude = ["Grazing Open","Grazing Forested","Forestry"]

FUsAsSQL = ""
count = 0
for thisFU in FUsToInclude:
    if(count == 0):
        FUsAsSQL = "'" + thisFU + "'"
    else:
        FUsAsSQL += ",'" + thisFU + "'"
    count += 1

FUsAsSQL

"'Grazing Open','Grazing Forested','Forestry','Conservation'"

In [17]:
lyr.SetAttributeFilter( FUField + " IN (" + FUsAsSQL + ")")
lyr.GetFeatureCount()

644

In [18]:
csvFile = rasterOutputFile.replace(".tif",".csv")
fumapper = pandas.DataFrame(columns=[uniqueIDField, catchmentField, FUField])
#featCount = 0
for feat in lyr:
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    #fumapper.append([feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)])
    newList = [feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)]
    fumapper.loc[len(fumapper)] = newList
    #featCount += 1
    #if (featCount == 20):
    #    break

#lyr.GetFeatureCount()
#fumapper.to_csv(csvFile, index=False)
#fumapper

In [19]:
##help(templateRas)
geo_transform = templateRas.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * templateRas.RasterXSize
y_min = y_max + geo_transform[5] * templateRas.RasterYSize
x_count = templateRas.RasterXSize
y_count = templateRas.RasterYSize
pixel_width = geo_transform[1]
pixel_height = geo_transform[5]

In [20]:
zone_ds = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, x_count, y_count, 1, gdal.GDT_Int32)
##Doing MEM did not yield the expected unique vals
##zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Byte)
# This was shifting the resulting raster northwards in our Albers stuff
#zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
band = zone_ds.GetRasterBand(1)
NoData_value = -9999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(zone_ds, [1], lyr, options=["ATTRIBUTE=" + uniqueIDField])

0

In [21]:
##This bit essential to get the raster file written - but could we use it before writing out (and then having to read back in?)
##Hopefully not needed if MEM type
##zone_ds = None
#type(zone_ds)

In [22]:
#zone_ds.GetGeoTransform()

In [23]:
zonesRasterArray = zone_ds.ReadAsArray()
zoneMask = numpy.ma.masked_where(zonesRasterArray == NoData_value, zonesRasterArray)
#uniq = numpy.unique(zonesRasterArray)
##print(uniq)


In [24]:
#type(lyr)
#stats = zonal_stats(lyr, kFactRas)

In [25]:
#K factor zonal stats
#Kfact_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, kFactRas, -9999)
# Will get 'converting a masked element to nan' error for tiny polygons

In [26]:
#Load LS factor zonal stats
#LSfact_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, lsFactRas, -9999)

In [27]:
#Fines raster zonal stats
#Fines_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, FinesRas, -9999)


In [28]:
#kFactRas = None
#lsFactRas = None
#FinesRas = None

#index = 0
#while index < len(Kfact_stats):
#    if(Kfact_stats[index][uniqueIDField] == 1199):
#        print(str(Kfact_stats[index][uniqueIDField]) + " KFactor Mean: " + str(Kfact_stats[index]['mean']) + " Count: " + str(Kfact_stats[index]['count']))
#        print(str(LSfact_stats[index][uniqueIDField]) + " LS Mean: " + str(LSfact_stats[index]['mean']) + " Count: " + str(LSfact_stats[index]['count']))
#        print(str(Fines_stats[index][uniqueIDField]) + " Fines Mean: " + str(Fines_stats[index]['mean']) + " Count: " + str(Fines_stats[index]['count']))
#        break
    
#    index += 1

In [29]:
#now need to combine K, LS & Fines

kFactNumpy = kFactRas.GetRasterBand(1).ReadAsArray()
lsFactNumpy = lsFactRas.GetRasterBand(1).ReadAsArray()
FinesNumpy = FinesRas.GetRasterBand(1).ReadAsArray()

kFactMasked = numpy.ma.masked_where(kFactNumpy == kFactRas.GetRasterBand(1).GetNoDataValue(), kFactNumpy)
lsFactMasked = numpy.ma.masked_where(lsFactNumpy == lsFactRas.GetRasterBand(1).GetNoDataValue(), lsFactNumpy)
FinesMasked = numpy.ma.masked_where(FinesNumpy == FinesRas.GetRasterBand(1).GetNoDataValue(), FinesNumpy)



kFactRas = None
lsFactRas = None
FinesRas = None




In [30]:
#what will this do for nodata?
# it will give -9999 ^ 3. Need to mask!
KLSFinesNumpy = kFactMasked * lsFactMasked * (FinesMasked / 100)
KLSNumpy = kFactMasked * lsFactMasked
print(KLSFinesNumpy.min())
#print(KLSFinesNumpy.max())


3.072e-05


In [31]:
fines_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [32]:
XCoordMid = fines_GeoTrans[0] + (25 * fines_GeoTrans[1]) + (fines_GeoTrans[1] / 2)
XCoordMid 

1678633.63051

In [33]:
YCoordMid = fines_GeoTrans[3] + (25 * fines_GeoTrans[5]) + (fines_GeoTrans[5] / 2)
YCoordMid

-2258725.4690071

In [34]:
theRowForX = (1678633.63051 - fines_GeoTrans[0] - (fines_GeoTrans[1] / 2)) / fines_GeoTrans[1]
theRowForX

25.0

In [35]:
theColForY = (-2258725.4690071 - fines_GeoTrans[3] - (fines_GeoTrans[5] / 2)) / fines_GeoTrans[5]
theColForY

25.0

In [36]:
def centreCoordsForPixel(gt, row, col):
    
    XCoordMid = gt[0] + (col * gt[1]) + (gt[1] / 2)
    YCoordMid = gt[3] + (row * gt[5]) + (gt[5] / 2)
    
    return (XCoordMid, YCoordMid)

In [37]:
def pixelForCentreCoords(gt, xCoord, yCoord):
    
    yCol = int((xCoord - gt[0] - (gt[1] / 2)) / gt[1])
    xRow = int((yCoord - gt[3] - (gt[5] / 2)) / gt[5])
    
    return(xRow, yCol)

In [38]:
tester = pixelForCentreCoords(fines_GeoTrans, 1678633.63051, -2258725.4690071)
tester

(25, 25)

In [39]:
test2 = centreCoordsForPixel(fines_GeoTrans, 33, 66)
test2[1]

-2258965.4690071

In [40]:
zone_ds.GetGeoTransform()


(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [41]:

def getDateFromFileName(theFileName, pattern):
    
    allowedChars = ["x", "y", "m", "d"]
    
    for cha in pattern:
        if cha not in allowedChars:
            print("You can't use '" + cha + "' ya dill")
            return
    
    
    if "yyyy" not in pattern:
        print("You've got to at least have 'yyyy' in your pattern")
        return
    
    Day = 1
    Month = 1
    Year = int(theFileName[pattern.index("yyyy"): pattern.index("yyyy") + 4])
    
    if "mm" in pattern:
        Month = int(theFileName[pattern.index("mm"): pattern.index("mm") + 2])
    
   
    if "dd" in pattern:
        Day = int(theFileName[pattern.index("dd"): pattern.index("dd") + 2])
        

    boogity = datetime.date(Year, Month, Day)
    
    return boogity

In [42]:
# list out Cfactor files
#cFactorDIR = "D:\MW\Ram_USLE\skip"

cRastersInfo = {}
cRastersAlone = []
for file in os.listdir(cFactorDIR):
    if file.endswith("." + cFactExt):
        #print(os.path.join(cFactorDIR, file) + " and " + str(getDateFromFileName(file, "xxxyyyymm")))
        #cRastersInfo.append(getDateFromFileName(file, "xxxyyyymm"), os.path.join(cFactorDIR, file))
        cRastersInfo[getDateFromFileName(file, cFactPattern)] =  os.path.join(cFactorDIR, file)
        cRastersAlone.append(os.path.join(cFactorDIR, file))

#cRastersInfo = sorted(cRastersInfo, reverse=True)
cRastersInfoKeysSorted = sorted(cRastersInfo)
print(cRastersInfoKeysSorted)

[datetime.date(1987, 9, 1), datetime.date(1987, 12, 1), datetime.date(1988, 3, 1), datetime.date(1988, 6, 1)]


In [81]:
#checkdate = cRastersInfoKeysSorted[0]

#print(cRastersInfo.get(checkdate))

#print(str(zonesRasterArray[100,100]))
#print(str(KLSNumpy[100,100]))
#print(str(zonesRasterArray[1000,1000]))
#print(str(NoData_value))

#Need to re-fill the masked KLS arrays so that we can detect 'nodata' cells
#mostly where our coastal mask is missing a slope cell or two
KLSNumpyFilled = numpy.ma.filled(KLSNumpy, NoData_value)
KLSFinesNumpyFilled = numpy.ma.filled(KLSFinesNumpy, NoData_value)

1
0.0140555
-9999
-9999


In [99]:
zoneIDStr = "zoneID"
zoneCountStr = "zoneCount"
validKLSCSum = "validKLSCSum"
allKLSCSum = "allKLSCSum"
validKLSCFinesSum = "validKLSCFinesSum"
allKLSCFinesSum = "allKLSCFinesSum"
validDataCountStr = "dataCount"

KLSCvalStr = "KLSC"
KLSCFinesValStr = "KLSCFines"

def analyseThisCFactorRaster(theCFactorRas, datePattern, theZonesRasterArray, zonesGT, 
                             ZonesRowCount, ZonesColCount, ZoneND, theDefaultC, theReqValidProp, 
                             theKLSNumpy, theKLSFinesNumpy):
    # do pixel by pixel analysis
    # zones raster of intersected FUs will exist, and KLSFines raster will be co-located with this
    # send back a dictionary that also includes the valid/invalid cFactor numbers for each FU
    # should be able to access rasters etc without passing them into this function
    # same for the dictionary of results, it could already exist
    
    # Will need to watch the Y min/max interpretation for our Aus Albers stuff
    zoneIDStr = "zoneID"
    zoneCountStr = "zoneCount"
    validKLSCSum = "validKLSCSum"
    allKLSCSum = "allKLSCSum"
    validKLSCFinesSum = "validKLSCFinesSum"
    allKLSCFinesSum = "allKLSCFinesSum"
    validDataCountStr = "dataCount"
    KLSCvalStr = "KLSC"
    KLSCFinesValStr = "KLSCFines"
    
    theRepDate = getDateFromFileName(os.path.basename(theCFactorRas), datePattern)

        
    cFactRas = gdal.Open(theCFactorRas)
    cFact_GeoTrans = cFactRas.GetGeoTransform()
    #print(cFact_GeoTrans)
    cFactND = cFactRas.GetRasterBand(1).GetNoDataValue()
    cFactNumpy = cFactRas.ReadAsArray()
    
    #print("CFactND: " + str(cFactND))
    
    cFactRows = cFactRas.RasterYSize
    cFactCols = cFactRas.RasterXSize
    
    #zGT = theZones.GetGeoTransform()
    zGT = zonesGT
    
    #theZonesRasterArray = theZones.ReadAsArray()
    

    statsDict = {}
    statsDictToReturn = {}
    uniq = numpy.unique(theZonesRasterArray)
    for num in uniq:
        #using masked, so this check for NoData should be redundant
        #print("trying this UID: " + str(num))
        if num == ZoneND:
            continue
        
        statsDict[num] = {zoneCountStr: int(0), validKLSCSum: float(0), validKLSCFinesSum: float(0),
                          allKLSCSum: float(0), allKLSCFinesSum: float(0), validDataCountStr: int(0)}

    rowid = 0
    while rowid < ZonesRowCount:
    #while rowid < theZones.RasterYSize:
        #reset colid
        colid = 0
        #if rowid % 500 == 0:
        #    print("Starting row: " + str(rowid))
        
        while colid < ZonesColCount:
        #while colid < theZones.RasterXSize:
            zoneVal = theZonesRasterArray[rowid, colid]
            
            if zoneVal == ZoneND:
                colid+=1
                continue
            
            #we have explicitly ensured that KLSFines matches our pixels
            #although we will have filled the mask with NoData vals
            #so just need to get the requisite cFactor row/col
            
            KLS = theKLSNumpy[rowid, colid]
            if KLS == ZoneND:
                colid+=1
                continue
            
            KLSFines = theKLSFinesNumpy[rowid, colid]
            if KLSFines == ZoneND:
                colid+=1
                continue
            
            
            zoneCoords = centreCoordsForPixel(zGT, rowid, colid)
            cFactPix = pixelForCentreCoords(cFact_GeoTrans, zoneCoords[0], zoneCoords[1])
            
            #if cFactPix[0] >= cFactRows:
            #    print("Exceeded row count at " + str(cFactPix[0]) + " for coords: " + str(zoneCoords)
            #          + " where zone pixels were: " + str(rowid) + ", " + str(colid))
            #    SystemExit(0)
            
            #if cFactPix[1] >= cFactCols:
            #    print("Exceeded col count " + str(cFactPix[1]) + " for coords: " + str(zoneCoords)
            #          + " where zone pixels were: " + str(rowid) + ", " + str(colid))
            #    SystemExit(0)
            
            theC = theDefaultC
            
            isValid = False
            cRasVal = cFactNumpy[cFactPix[0], cFactPix[1]]
            if not cRasVal == cFactND:
                theC = cRasVal
                isValid = True
            
            KLSC = KLS * theC
            KLSCFines = KLSFines * theC
            
            #if(KLSC < 0):
            #    print("Badness at "  + str(rowid) + ", " + str(colid) + " where KLS was: " 
            #          + str(theKLSNumpy[rowid, colid]) + " and C was: " + str(theC))
            #    SystemExit(0)
            
            statsDict[zoneVal][zoneCountStr] += 1
            statsDict[zoneVal][allKLSCSum] += KLSC
            statsDict[zoneVal][allKLSCFinesSum] += KLSCFines
            
            if(isValid):
                statsDict[zoneVal][validKLSCSum] += KLSC
                statsDict[zoneVal][validKLSCFinesSum] += KLSCFines
                statsDict[zoneVal][validDataCountStr] += 1
            
            
            
            colid += 1
        
        
        rowid += 1
        
        
    #now work out which stats to use
    #look to see if we'll use the default Cfactor derived values in lieu of valid entries
    for uid in statsDict.keys():
        
        #if uid == 34:
        #    print(statsDict[uid])
        
        zoneCount = statsDict[uid][zoneCountStr]
        if zoneCount == 0:
            #nothing to report, add zeros
            statsDictToReturn[uid] = {KLSCvalStr: -9999, KLSCFinesvalStr: -9999}
            continue
        
        valProp = 0
        valCount = statsDict[uid][validDataCountStr]
        if(valCount > 0):
            valProp = valCount / zoneCount
         
        if valProp >= theReqValidProp:
            # use valid only stats
            statsDictToReturn[uid] = {KLSCvalStr: float(statsDict[uid][validKLSCSum] / valCount),
                                      KLSCFinesValStr: float(statsDict[uid][validKLSCFinesSum] / valCount)}
        else:
            # Use stats that include the default where necessary
            statsDictToReturn[uid] = {KLSCvalStr: float(statsDict[uid][allKLSCSum] / zoneCount),
                                      KLSCFinesValStr: float(statsDict[uid][allKLSCFinesSum] / zoneCount)}
        
    
    statsWithDate = {}
    statsWithDate[theRepDate] = statsDictToReturn
    
    theCFatcorRas = None
    theZonesRasterArray = None
    zonesGT = None
    ZonesRowCount =  None
    ZonesColCount = None
    ZoneND = None
    theDefaultC = None
    theReqValidProp = None
    theKLSNumpy = None
    theKLSFinesNumpy = None
    
    gc.collect()
    
    return statsWithDate

In [100]:
#start_time = time.time()


#tester = analyseThisCFactorRaster("D:\\MW\\Ram_USLE\\skip\\justone\\cf_198712_198802.tif", 
#                                  zonesRasterArray, zone_ds.GetGeoTransform(), 
#                                           zone_ds.RasterYSize, zone_ds.RasterXSize, NoData_value, defaultCfact, 
#                                           reqdValidProp, KLSNumpyFilled, KLSFinesNumpyFilled)



#elapsed_time = time.time() - start_time
#print("Finished One: " + str(len(tester)) + " taking this many seconds: " + str(elapsed_time))

In [101]:
#print(tester)

In [103]:
#%load usle_cfactor_analysis

start_time = time.time()

usleStats = {}

mydict=dict(theZonesRasterArray=zonesRasterArray, datePattern=cFactPattern, zonesGT=zone_ds.GetGeoTransform(), ZonesRowCount=zone_ds.RasterYSize, 
            ZonesColCount=zone_ds.RasterXSize, ZoneND=NoData_value, 
            theDefaultC=defaultCfact, theReqValidProp=reqdValidProp, 
            theKLSNumpy=KLSNumpyFilled, theKLSFinesNumpy=KLSFinesNumpyFilled)

#print(mydict)

#num_cores = multiprocessing.cpu_count()
#print(str(num_cores) + " cores available")

#xxx = lambda cRas:usle_cfactor_analysis.analyseThisCFactorRaster(cRas,
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp)

#xxx = lambda cRas:analyseThisCFactorRaster(cRas, zonesRasterArray, zone_ds.GetGeoTransform(), 
#                                           zone_ds.RasterYSize, zone_ds.RasterXSize, 
#                                           NoData_value, defaultCfact, reqdValidProp)

xxx = lambda theCFactorRas:analyseThisCFactorRaster(theCFactorRas, datePattern, theZonesRasterArray, zonesGT, 
                                           ZonesRowCount, ZonesColCount, ZoneND, theDefaultC, 
                                           theReqValidProp, theKLSNumpy, theKLSFinesNumpy)


#xxx = lambda rasDate:usle_cfactor_analysis.analyseThisCFactorRaster(cRastersInfo.get(rasDate),
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp)

print("doing execute")
#rc[:].execute('import D:/PythonScriptingForGBR/reef_scripts/GBR/Examples/Modules/usle_cfactor_analysis')
rc[:].execute("import numpy")
rc[:].execute("import gc")
rc[:].execute("import os")
rc[:].execute("import datetime")
rc[:].execute("from osgeo import gdal, gdalconst")
rc[:].execute("from osgeo import ogr")

print("doing push")
rc[:].push({"analyseThisCFactorRaster":analyseThisCFactorRaster})
rc[:].push({"centreCoordsForPixel":centreCoordsForPixel})
rc[:].push({"pixelForCentreCoords":pixelForCentreCoords})
rc[:].push({"getDateFromFileName":getDateFromFileName})
print("starting pushing dict")
rc[:].push(mydict)
print("finished pushing dict")
results = rc[:].map_sync(xxx, cRastersAlone)
#results = rc[:].map_sync(xxx, cRastersInfoKeysSorted)

#results = rc[:].map_sync(lambda rasDate: usle_cfactor_analysis.analyseThisCFactorRaster(cRastersInfo.get(rasDate),
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp), 
#                         cRastersInfoKeysSorted)




#results = Parallel(n_jobs=num_cores)(delayed(analyseThisCFactorRaster)(cRastersInfo.get(rasDate), zonesRasterArray, zone_ds.RasterYSize, zone_ds.RasterXSize, NoData_value, defaultCfact, reqdValidProp) 
#                                     for rasDate in cRastersInfoKeysSorted)



#for rasDate in cRastersInfoKeysSorted:
    #Process the cFator raster
    # processedStats will have the date as primary key and then each Unique FU ID and some relevant stats
#    start_time = time.time()
#    print("starting work on " + str(cRastersInfo.get(rasDate)))
    
#    processedStats = analyseThisCFactorRaster(cRastersInfo.get(rasDate))
    
#    elapsed_time = time.time() - start_time
    
#    print("finished work on " + str(cRastersInfo.get(rasDate)) + " taking " + str(elapsed_time) + " seconds")
#    print("this big " + str(len(processedStats)))
    
    
#    #Add results to the stats
#    usleStats[rasDate] = processedStats
    
elapsed_time = time.time() - start_time

print("Got this many rasters processed: " + str(len(results)) + " taking this many seconds: " + str(elapsed_time))

doing execute
doing push
starting pushing dict
finished pushing dict
Got this many rasters processed: 4 taking this many seconds: 307.91256284713745


In [152]:
#type(results[0])
#results[0].keys()
convertedDict = {}
availDates = []
for gg in results:
    print(str(gg.keys()) + " and " + str(len(gg.keys())))
    #Know that we only have one key, but need to loop anyway
    for dateKey in gg.keys():
        convertedDict[dateKey] = gg.values()
        availDates.append(dateKey)
    #convertedDict[gg.keys()[0]] = gg.values()


print(len(convertedDict))
sortedAvailDates = sorted(availDates)
#convertedDict

dict_keys([datetime.date(1987, 9, 1)]) and 1
dict_keys([datetime.date(1987, 12, 1)]) and 1
dict_keys([datetime.date(1988, 3, 1)]) and 1
dict_keys([datetime.date(1988, 6, 1)]) and 1
4


In [153]:
#convertedDict[datetime.date(1988, 3, 1)]
convertedDict.keys()

dict_keys([datetime.date(1988, 3, 1), datetime.date(1987, 9, 1), datetime.date(1987, 12, 1), datetime.date(1988, 6, 1)])

In [154]:
# now need FU-based KLSC & KLSCFines averages for tempral gap filling
avgs = {}
unZones = numpy.unique(zonesRasterArray)
klscSumStr = "KLSCSum"
klscfinesSumStr = "KLSCFinesSum"
klscCountStr = "KLSCCount"
klscfinesCountStr = "KLSCFinesCount"

for zn in unZones:
    avgs[zn] = {klscSumStr: float(0), klscCountStr: int(0), 
                klscfinesSumStr: float(0), klscfinesCountStr: int(0)}

# now populate
for dt in convertedDict.keys():
    #print(type(convertedDict[dt]))
    #print(convertedDict[dt])
    for fuDict in convertedDict[dt]:
        #print(type(fuDict))
        for fuid in fuDict.keys():
            #print(fuDict[fuid])
            
            if not fuDict[fuid][KLSCvalStr] == -9999:
                avgs[fuid][klscSumStr] += fuDict[fuid][KLSCvalStr]
                avgs[fuid][klscCountStr] += 1
            if not fuDict[fuid][KLSCFinesValStr] == -9999:
                avgs[fuid][klscfinesSumStr] += fuDict[fuid][KLSCFinesValStr]
                avgs[fuid][klscfinesCountStr] += 1

print(avgs[3])
print(avgs[34])
print(avgs[1199])

{'KLSCFinesSum': 0.0026435113233422847, 'KLSCCount': 4, 'KLSCSum': 0.013025556641159087, 'KLSCFinesCount': 4}
{'KLSCFinesSum': 0.0064365387147866104, 'KLSCCount': 4, 'KLSCSum': 0.013341741968894552, 'KLSCFinesCount': 4}
{'KLSCFinesSum': 0.0008726808767199328, 'KLSCCount': 4, 'KLSCSum': 0.0018440991843308753, 'KLSCFinesCount': 4}


In [156]:
# would now need to determine the required timespan for output timeseries
# grabbing relevant values form either real or temporal average as needed

reqdStart = datetime.date(1987, 1, 1)
reqdLast = datetime.date(1990, 12, 31)

earliestAvail = sortedAvailDates[0]
latestAvail = sortedAvailDates[len(sortedAvailDates) - 1]

print(earliestAvail)
print(latestAvail)

1987-09-01
1988-06-01


In [157]:
if reqdStart < earliestAvail:
    print("Defs need padding at start")
else:
    print("No start padding required")

Defs need padding at start
